In [6]:
import pandas as pd
import torch
import torch.nn as nn

# MLPモデルクラスの定義
class MLPModel(nn.Module):
    def __init__(self):
        super(MLPModel, self).__init__()
        self.fc1 = nn.Linear(4, 64) 
        self.fc2 = nn.Linear(64, 32) 
        self.fc3 = nn.Linear(32, 1)  
        
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        x = self.sigmoid(x)
        return x

# データファイルとモデルファイルのパス
csv_path = "./csv/sdxl-turbo_merged.csv"
model_path = "./MlpNonLinearPredictor.pth"

# CSVファイルを読み込み
print("Loading CSV file...")
data = pd.read_csv(csv_path)
data = data.dropna()

# 特徴量の選択
features = data[['arniqa', 'clipiqa', 'cnniqa', 'metaiqa']]

# 特徴量をTensorに変換
features_tensor = torch.tensor(features.values, dtype=torch.float32)

# モデルの読み込み
print("Loading pre-trained model...")
model = MLPModel()
model = torch.load(model_path)
model.eval()

# 推論の実行
print("Running inference...")
with torch.no_grad():
    overall_scores = model(features_tensor).squeeze().numpy()

# 結果をDataFrameに追加
data['overall_score'] = overall_scores

# 更新されたCSVファイルを保存
output_csv_path = "./csv/sdxl-turbo_with_overall_score.csv"
data.to_csv(output_csv_path, index=False)

print(f"Updated CSV file saved to: {output_csv_path}")


Loading CSV file...
Loading pre-trained model...
Running inference...
Updated CSV file saved to: ./csv/sdxl-turbo_with_overall_score.csv


/tmp/ipykernel_508415/2679531189.py:41: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_path)
